In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
from shutil import copy2
import csv
import tensorflow as tf
import pandas as pd

So first off, we set the path to our dataset in `daat_root`

In [3]:
data_root = ("/content/drive/MyDrive/Dataset")

Now I am going to split the image data in the `data_root`  into two sets, i.e., training and validation sets

In [22]:
#Validation image data
IMAGE_SHAPE = (224, 224)
TRAINING_DATA_DIR = str(data_root)
datagen_kwargs = dict(rescale=1./255, validation_split=.20)
valid_datagen = tf.keras.preprocessing.image.ImageDataGenerator(**datagen_kwargs)
valid_generator = valid_datagen.flow_from_directory(
    data_root,
    subset="validation",
    shuffle=True,
    target_size=IMAGE_SHAPE,
)



#Training image data
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(**datagen_kwargs)
train_generator = train_datagen.flow_from_directory(
    data_root,
    subset="training",
    shuffle=True,
    target_size=IMAGE_SHAPE
    #batch_size = 10 #default 32
    )







Found 100 images belonging to 5 classes.
Found 400 images belonging to 5 classes.


As can be seen above, at this pont I have 100 images set for validation and 400 for training. Now at this point, I would like to split the validation data into further two sets i.e., validation and test sets (50 images for each set in this example)

In [23]:
validation_images = valid_generator.filenames
validation_images

['class_1/image_001.jpg',
 'class_1/image_002.jpg',
 'class_1/image_003.jpg',
 'class_1/image_004.jpg',
 'class_1/image_005.jpg',
 'class_1/image_006.jpg',
 'class_1/image_007.jpg',
 'class_1/image_008.jpg',
 'class_1/image_009.jpg',
 'class_1/image_010.jpg',
 'class_1/image_011.jpg',
 'class_1/image_012.jpg',
 'class_1/image_013.jpg',
 'class_1/image_014.jpg',
 'class_1/image_015.jpg',
 'class_1/image_016.jpg',
 'class_1/image_017.jpg',
 'class_1/image_018.jpg',
 'class_1/image_019.jpg',
 'class_1/image_020.jpg',
 'class_2/image_001.jpg',
 'class_2/image_002.jpg',
 'class_2/image_003.jpg',
 'class_2/image_004.jpg',
 'class_2/image_005.jpg',
 'class_2/image_006.jpg',
 'class_2/image_007.jpg',
 'class_2/image_008.jpg',
 'class_2/image_009.jpg',
 'class_2/image_010.jpg',
 'class_2/image_011.jpg',
 'class_2/image_012.jpg',
 'class_2/image_013.jpg',
 'class_2/image_014.jpg',
 'class_2/image_015.jpg',
 'class_2/image_016.jpg',
 'class_2/image_017.jpg',
 'class_2/image_018.jpg',
 'class_2/im

In [24]:
validation_labels = valid_generator.classes
validation_labels

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4], dtype=int32)

In [25]:
#Now I need to turn the integer class names shown above into strings for the code in below cells to work
dummy=['class_1', 'class_2', 'class_2', 'class_3', 'class_4']
TL=[]
for i in range(len(validation_labels)):
  TL.append(dummy[validation_labels[i]])
validation_labels=TL
validation_labels

['class_1',
 'class_1',
 'class_1',
 'class_1',
 'class_1',
 'class_1',
 'class_1',
 'class_1',
 'class_1',
 'class_1',
 'class_1',
 'class_1',
 'class_1',
 'class_1',
 'class_1',
 'class_1',
 'class_1',
 'class_1',
 'class_1',
 'class_1',
 'class_2',
 'class_2',
 'class_2',
 'class_2',
 'class_2',
 'class_2',
 'class_2',
 'class_2',
 'class_2',
 'class_2',
 'class_2',
 'class_2',
 'class_2',
 'class_2',
 'class_2',
 'class_2',
 'class_2',
 'class_2',
 'class_2',
 'class_2',
 'class_2',
 'class_2',
 'class_2',
 'class_2',
 'class_2',
 'class_2',
 'class_2',
 'class_2',
 'class_2',
 'class_2',
 'class_2',
 'class_2',
 'class_2',
 'class_2',
 'class_2',
 'class_2',
 'class_2',
 'class_2',
 'class_2',
 'class_2',
 'class_3',
 'class_3',
 'class_3',
 'class_3',
 'class_3',
 'class_3',
 'class_3',
 'class_3',
 'class_3',
 'class_3',
 'class_3',
 'class_3',
 'class_3',
 'class_3',
 'class_3',
 'class_3',
 'class_3',
 'class_3',
 'class_3',
 'class_3',
 'class_4',
 'class_4',
 'class_4',
 'cl

In [26]:
from sklearn.model_selection import train_test_split
val_set1_images, val_set2_images, val_set1_labels, val_set2_labels = train_test_split(
    validation_images,
    validation_labels,
    test_size=0.5,
    random_state=42)


In [27]:
  dataframe=pd.DataFrame({'filename': val_set1_images, 'class': val_set1_labels})
  dataframe

,filename,class
0,class_2/image_005.jpg,class_2
1,class_1/image_004.jpg,class_1
2,class_1/image_018.jpg,class_1
3,class_2/image_019.jpg,class_2
4,class_1/image_009.jpg,class_1
5,class_4/image_019.jpg,class_3
6,class_1/image_007.jpg,class_1
7,class_4/image_005.jpg,class_3
8,class_2/image_017.jpg,class_2
9,class_5/image_010.jpg,class_4


In [28]:
#val_datagen1 is for validation while val_datagen2 is for testing
val_datagen1 = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
val_datagen2 = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

val_gen_1 = val_datagen1.flow_from_dataframe(
    dataframe=pd.DataFrame({'filename': val_set1_images, 'class': val_set1_labels}),
    directory=data_root,
    x_col='filename',
    y_col='class',
    target_size=IMAGE_SHAPE,
)

val_gen_2 = val_datagen2.flow_from_dataframe(
    dataframe=pd.DataFrame({'filename': val_set2_images, 'class': val_set2_labels}),
    directory=data_root,
    x_col='filename',
    y_col='class',
    target_size=IMAGE_SHAPE,
)


Found 50 validated image filenames belonging to 4 classes.
Found 50 validated image filenames belonging to 4 classes.


In [30]:
val_gen_1.filenames

['class_2/image_005.jpg',
 'class_1/image_004.jpg',
 'class_1/image_018.jpg',
 'class_2/image_019.jpg',
 'class_1/image_009.jpg',
 'class_4/image_019.jpg',
 'class_1/image_007.jpg',
 'class_4/image_005.jpg',
 'class_2/image_017.jpg',
 'class_5/image_010.jpg',
 'class_3/image_017.jpg',
 'class_5/image_020.jpg',
 'class_3/image_015.jpg',
 'class_3/image_004.jpg',
 'class_3/image_011.jpg',
 'class_4/image_008.jpg',
 'class_3/image_007.jpg',
 'class_4/image_009.jpg',
 'class_4/image_002.jpg',
 'class_5/image_018.jpg',
 'class_4/image_020.jpg',
 'class_3/image_002.jpg',
 'class_3/image_019.jpg',
 'class_3/image_009.jpg',
 'class_5/image_019.jpg',
 'class_3/image_018.jpg',
 'class_4/image_016.jpg',
 'class_2/image_013.jpg',
 'class_5/image_015.jpg',
 'class_3/image_020.jpg',
 'class_4/image_004.jpg',
 'class_5/image_005.jpg',
 'class_2/image_018.jpg',
 'class_2/image_010.jpg',
 'class_1/image_002.jpg',
 'class_3/image_013.jpg',
 'class_2/image_002.jpg',
 'class_1/image_003.jpg',
 'class_2/im

In [19]:
val_gen_1.labels

[1,
 0,
 0,
 1,
 0,
 2,
 0,
 2,
 1,
 3,
 1,
 3,
 1,
 1,
 1,
 2,
 1,
 2,
 2,
 3,
 2,
 1,
 1,
 1,
 3,
 1,
 2,
 1,
 3,
 1,
 2,
 3,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 3,
 3,
 2,
 3,
 3,
 1,
 2,
 2,
 0,
 3,
 1]

In [20]:
val_gen_2.filenames

['class_5/image_004.jpg',
 'class_3/image_014.jpg',
 'class_4/image_011.jpg',
 'class_3/image_006.jpg',
 'class_3/image_005.jpg',
 'class_2/image_020.jpg',
 'class_2/image_003.jpg',
 'class_5/image_001.jpg',
 'class_1/image_011.jpg',
 'class_1/image_001.jpg',
 'class_1/image_019.jpg',
 'class_2/image_011.jpg',
 'class_4/image_014.jpg',
 'class_2/image_014.jpg',
 'class_5/image_011.jpg',
 'class_1/image_005.jpg',
 'class_4/image_017.jpg',
 'class_4/image_018.jpg',
 'class_1/image_013.jpg',
 'class_2/image_012.jpg',
 'class_3/image_016.jpg',
 'class_5/image_009.jpg',
 'class_2/image_007.jpg',
 'class_3/image_003.jpg',
 'class_4/image_010.jpg',
 'class_1/image_016.jpg',
 'class_3/image_001.jpg',
 'class_5/image_017.jpg',
 'class_1/image_010.jpg',
 'class_4/image_013.jpg',
 'class_1/image_012.jpg',
 'class_3/image_008.jpg',
 'class_5/image_006.jpg',
 'class_2/image_009.jpg',
 'class_5/image_014.jpg',
 'class_1/image_006.jpg',
 'class_4/image_007.jpg',
 'class_4/image_006.jpg',
 'class_2/im

In [21]:
val_gen_2.labels

[3,
 1,
 2,
 1,
 1,
 1,
 1,
 3,
 0,
 0,
 0,
 1,
 2,
 1,
 3,
 0,
 2,
 2,
 0,
 1,
 1,
 3,
 1,
 1,
 2,
 0,
 1,
 3,
 0,
 2,
 0,
 1,
 3,
 1,
 3,
 0,
 2,
 2,
 1,
 0,
 1,
 1,
 0,
 3,
 1,
 0,
 3,
 1,
 2,
 0]

#The image data is now ready to be fed to a Deep Learning model created with Keras